In [1]:
# import os.path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

# # If modifying these scopes, delete the file token.json.
# SCOPES = ["https://www.googleapis.com/auth/youtube.readonly","https://www.googleapis.com/auth/youtube.force-ssl"]


# creds = None
# # The file token.json stores the user's access and refresh tokens, and is
# # created automatically when the authorization flow completes for the first
# # time.
# if os.path.exists("token.json"):
#   creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# # If there are no (valid) credentials available, let the user log in.
# if not creds or not creds.valid:
#   if creds and creds.expired and creds.refresh_token:
#     creds.refresh(Request())
#   else:
#     flow = InstalledAppFlow.from_client_secrets_file(
#         "credentials.json", SCOPES
#     )
#     creds = flow.run_local_server(port=0)
#   # Save the credentials for the next run
#   with open("token.json", "w") as token:
#     token.write(creds.to_json())

# try:
#   youtube = build("youtube", "v3", credentials=creds)

#       # Taking input from the user and slicing for video id
#   video_id = input('Enter Youtube Video URL: ')[-11:]
#   print("video id: " + video_id)
  
#   # Getting the channelId of the video uploader
#   video_response = youtube.videos().list(
#       part='snippet',
#       id=video_id
#   ).execute()
  
#   print(video_response)
#   # Splitting the response for channelID
#   video_snippet = video_response['items'][0]['snippet']
#   uploader_channel_id = video_snippet['channelId']
#   print("channel id: " + uploader_channel_id)
  
# except HttpError as err:
#   print(err)


In [29]:
from googleapiclient.discovery import build
import re

API_KEY = ''  # Put in your API Key

youtube = build('youtube', 'v3', developerKey=API_KEY)  # initializing Youtube API

# Improved input handling for video ID
def extract_video_id(url):
    match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    return match.group(1) if match else None

video_url = input('Enter YouTube Video URL: ')
video_id = extract_video_id(video_url)

if not video_id:
    print("Invalid YouTube URL")
else:
    print("video id: " + video_id)

    # Getting the channelId of the video uploader
    try:
        video_response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()

        print("Video Response:", video_response)  # Debugging step

        if 'items' in video_response and video_response['items']:
            video_snippet = video_response['items'][0]['snippet']
            uploader_channel_id = video_snippet.get('channelId', 'No channelId found')
            print("channel id: " + uploader_channel_id)

            # Fetch comments
            print("Fetching Comments...")
            comments = []
            nextPageToken = None
            while len(comments) < 600:
                request = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=100,  # You can fetch up to 100 comments per request
                    pageToken=nextPageToken
                )
                response = request.execute()
                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    # Check if the comment is not from the video uploader
                    if comment['authorChannelId']['value'] != uploader_channel_id:
                        comments.append(comment['textDisplay'])
                nextPageToken = response.get('nextPageToken')

                if not nextPageToken:
                    break
            # Print the first 5 comments
            print(comments[:5])

        else:
            print("No video found for the provided ID.")
    except Exception as e:
        print("An error occurred:", e)


video id: oWo9SNcyxlI
Video Response: {'kind': 'youtube#videoListResponse', 'etag': 'FDRV--UbntAuYJOaIEc1BeeHFCo', 'items': [{'kind': 'youtube#video', 'etag': 'ppDsgJg80plQ9aIWXXT-RcV1qtM', 'id': 'oWo9SNcyxlI', 'snippet': {'publishedAt': '2022-07-27T13:15:20Z', 'channelId': 'UC8UmuMDlWB1egqH5nyy0pDw', 'title': 'Sentiment Analysis with LSTM | Deep Learning with Keras | Neural Networks | Project#8', 'description': 'We shall train three separate Neural Networks, namely: a Simple Neural Net, a Convolutional Neural Net and a Long Short Term Memory Neural Net. LSTM networks are actually considered to be quite suitable for handling NLP problems, and by the end of this video, you will understand why.\n\nSentiment Classification LSTM Model we shall be training as part of this tutorial is so good that it not only predicts the user movie review sentiment as positive/negative, it also does a fabulous job in predicting IMDb Rating itself corresponding to these reviews with mind blowing accuracy.\n\

# Channel to CsV

In [30]:
import csv
from googleapiclient.discovery import build
import time

# Get the channel name and upload playlist ID
def get_channel_info(channel_id):
    response = youtube.channels().list(
        part='snippet,contentDetails',
        id=channel_id
    ).execute()
    if 'items' in response and len(response['items']) > 0:
        channel = response['items'][0]['snippet']['title']
        upload_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        return channel, upload_playlist_id
    return None, None

# Fetch 100 video IDs from the uploads playlist
def get_video_ids_from_channel(upload_playlist_id, max_results=100):
    video_ids = []
    next_page_token = None

    while len(video_ids) < max_results:
        response = youtube.playlistItems().list(
            part='snippet',
            playlistId=upload_playlist_id,
            maxResults=50,  # API allows max 50 results per request
            pageToken=next_page_token
        ).execute()
        
        for item in response['items']:
            video_ids.append(item['snippet']['resourceId']['videoId'])

        next_page_token = response.get('nextPageToken')
        if not next_page_token or len(video_ids) >= max_results:
            break
    
    return video_ids[:max_results]

# Fetch the video title
def get_video_title(video_id):
    response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()
    if 'items' in response and len(response['items']) > 0:
        return response['items'][0]['snippet']['title']
    return None

# Fetch comments for a video
def get_comments(video_id, max_comments=100):
    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,  # Fetch up to 100 comments per request
            pageToken=next_page_token
        ).execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        next_page_token = response.get('nextPageToken')
        if not next_page_token or len(comments) >= max_comments:
            break

    return comments[:max_comments]

# Write the data to a CSV file
def write_to_csv(channel_name, video_data):
    with open(f'{channel_name}_video_comments.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Channel Name", "Video Title", "Comment"])
        
        for video_title, comments in video_data.items():
            for comment in comments:
                writer.writerow([channel_name, video_title, comment])

# Main Function
def fetch_videos_and_comments(channel_id, max_videos=100, max_comments_per_video=100):
    start_time = time.time()
    # Get channel information
    channel_name, upload_playlist_id = get_channel_info(channel_id)
    
    if not channel_name or not upload_playlist_id:
        print("Invalid channel ID or channel information not found.")
        return

    print(f"Fetching data for channel: {channel_name}")

    # Get video IDs
    video_ids = get_video_ids_from_channel(upload_playlist_id, max_results=max_videos)

    video_data = {}

    # For each video, get the title and comments
    for video_id in video_ids:
        video_title = get_video_title(video_id)
        if video_title:
            print(f"Fetching comments for video: {video_title}")
            comments = get_comments(video_id, max_comments=max_comments_per_video)
            video_data[video_title] = comments
    
    
    # Write all data to a CSV
    write_to_csv(channel_name, video_data)
    print(f"Data saved to {channel_name}_video_comments.csv")
    end_time = time.time()
    execution_time = start_time - end_time
    print("Execution time:",execution_time)



In [31]:
import time
start_time = time.time()
# Replace with the actual YouTube channel ID
channel_id = f'UCX6OQ3DkcsbYNE6H8uQQuVA'
fetch_videos_and_comments(channel_id, max_videos=100, max_comments_per_video=100)


Fetching data for channel: MrBeast
Fetching comments for video: How Many Twins Can You Spot?
Fetching comments for video: Spot The Difference, Win $10,000
Fetching comments for video: 100 Identical Twins Fight For $250,000
Fetching comments for video: Running With Bigger And Bigger Lunchlys
Fetching comments for video: Holding Bigger And Bigger Dogs
Fetching comments for video: Men Vs Women Survive The Wilderness For $500,000
Fetching comments for video: Will A Guitar Boat Hold My Weight?
Fetching comments for video: Real Or Cake For $10,000
Fetching comments for video: 7 Days Stranded In A Cave
Fetching comments for video: Running With Bigger And Bigger Feastables
Fetching comments for video: Survive 100 Days In Nuclear Bunker, Win $500,000
Fetching comments for video: Spot The Fake Animal For $10,000
Fetching comments for video: 50 YouTubers Fight For $1,000,000
Fetching comments for video: Pass The Phone To…
Fetching comments for video: How Many Balloons Does It Take To Fly?
Fetchin

## Add Processing for data

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# is_cuda = torch.cuda.is_available()

# # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
# if is_cuda:
#     device = torch.device("cuda")
#     print("GPU is available")
# else:
#     device = torch.device("cpu")
#     print("GPU not available, CPU used")

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Load the CSV file
df = pd.read_csv('MrBeast_video_comments.csv', encoding='utf-8')

# Display the first few rows and basic information about the dataset
print(df.head())
print(df.info())

# Preprocess the data
# Use the 'Comment' column as our input and 'Video Title' as our target
X = df['Comment'].values
y = df['Video Title'].values

  Channel Name                              Video Title  \
0      MrBeast  Running With Bigger And Bigger Lunchlys   
1      MrBeast  Running With Bigger And Bigger Lunchlys   
2      MrBeast  Running With Bigger And Bigger Lunchlys   
3      MrBeast  Running With Bigger And Bigger Lunchlys   
4      MrBeast  Running With Bigger And Bigger Lunchlys   

                                  Comment  
0                              I love you  
1  Please give me one iPhone😢😢😢😢!🥺🥺🥺🥺🥺🥺🥺🥺  
2       ❤❤😂😂🎉😢😮😮😅😊😊❤❤❤😂😂😂🎉🎉😢😢😢😮😮😅😅😊😊❤❤😂😂😢  
3        It&#39;s a lot it&#39;s all good  
4         Can you buy a car for my dad ??  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Channel Name  9800 non-null   object
 1   Video Title   9800 non-null   object
 2   Comment       9800 non-null   object
dtypes: object(3)
memory usage: 229.8+ KB
None


In [20]:
import torch
from emoji import demojize


# Update the preprocessing function to include emoji conversion
def preprocess_text_with_emojis(text):
    # Convert emojis to text
    
    emoji_pattern = re.compile(r':([a-zA-Z_]+):')
    unique_emojis = set(emoji_pattern.findall(text))
    
    # Reconstruct the text with only one instance of each emoji
    for emote in unique_emojis:
        text = re.sub(f'(:{emote}:)+', f':{emote}:', text)

    # Convert to lowercase
    text = text.lower()
    # Remove special characters, numbers, and extra whitespace
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Reprocess the comments with the updated function
df['processed_comment_with_emojis'] = df['Comment'].apply(preprocess_text_with_emojis)

print("\
First 5 processed comments with emojis:")

print(df['processed_comment_with_emojis'].head())

def preprocess_comment(comment):
    # Replace emojis with their meanings
    return demojize(comment, language='en')

# Preprocess the comments
df['Preprocessed_Comment'] = df['Comment'].apply(preprocess_comment)

# Display the first few rows of the preprocessed data
print(df[['Comment', 'Preprocessed_Comment']].head())

# Convert preprocessed comments to PyTorch tensors
comment_tensors = [torch.tensor([ord(c) for c in comment]) for comment in df['Preprocessed_Comment']]

# Display the first tensor
print("\
First comment tensor:")
print(comment_tensors[0])

# Print the shape of the first tensor
print("\
Shape of the first tensor:", comment_tensors[0].shape)

# Print the total number of tensors
print("\
Total number of tensors:", len(comment_tensors))

First 5 processed comments with emojis:
0                      i love you
1       please give me one iphone
2                                
3          its a lot its all good
4    can you buy a car for my dad
Name: processed_comment_with_emojis, dtype: object
                                  Comment  \
0                              I love you   
1  Please give me one iPhone😢😢😢😢!🥺🥺🥺🥺🥺🥺🥺🥺   
2       ❤❤😂😂🎉😢😮😮😅😊😊❤❤❤😂😂😂🎉🎉😢😢😢😮😮😅😅😊😊❤❤😂😂😢   
3        It&#39;s a lot it&#39;s all good   
4         Can you buy a car for my dad ??   

                                Preprocessed_Comment  
0                                         I love you  
1  Please give me one iPhone:crying_face::crying_...  
2  :red_heart::red_heart::face_with_tears_of_joy:...  
3                   It&#39;s a lot it&#39;s all good  
4                    Can you buy a car for my dad ??  
First comment tensor:
tensor([ 73,  32, 108, 111, 118, 101,  32, 121, 111, 117])
Shape of the first tensor: torch.Size([10])
Total number 

In [26]:
import pandas as pd
import re
from emoji import demojize

def simple_sentiment_analysis(text):
    text = demojize(text.lower())
    
    positive_words = ['love', 'great', 'awesome', 'amazing', 'good', 'best', 'like', 'thanks', 'thank you', ':smiling_face:', ':red_heart:']
    negative_words = ['hate', 'bad', 'terrible', 'worst', 'dislike', 'awful', ':angry_face:', ':crying_face:']
    
    positive_count = sum(1 for word in positive_words if word in text)
    negative_count = sum(1 for word in negative_words if word in text)
    
    if positive_count > negative_count:
        return 'Positive'
    elif negative_count > positive_count:
        return 'Negative'
    else:
        return 'Neutral'

# Load the CSV file
df = pd.read_csv('MrBeast_video_comments.csv', encoding='utf-8')

# Apply sentiment analysis to all comments
df['Sentiment'] = df['Comment'].apply(simple_sentiment_analysis)

# Display the first few rows with sentiments
print(df[['Comment', 'Sentiment']].head(10))

# Display sentiment distribution
print("\
Sentiment Distribution:")
print(df['Sentiment'].value_counts(normalize=True))

# Save the labeled data
df.to_csv('MrBeast_video_comments_labeled.csv', index=False)
print("\
Labeled data saved to 'MrBeast_video_comments_labeled.csv'")

                                  Comment Sentiment
0                              I love you  Positive
1  Please give me one iPhone😢😢😢😢!🥺🥺🥺🥺🥺🥺🥺🥺  Negative
2       ❤❤😂😂🎉😢😮😮😅😊😊❤❤❤😂😂😂🎉🎉😢😢😢😮😮😅😅😊😊❤❤😂😂😢   Neutral
3        It&#39;s a lot it&#39;s all good  Positive
4         Can you buy a car for my dad ??   Neutral
5                        😅😂😂😂😂😂😂😂😮😂😮😮😂😮😂😂   Neutral
6                                      😂❤  Positive
7                           ❤❤❤❤ nice 👍👍👍  Positive
8                                   😂😂😂😂😂   Neutral
9                                      ❤❤  Positive
Sentiment Distribution:
Sentiment
Neutral     0.737245
Positive    0.223980
Negative    0.038776
Name: proportion, dtype: float64
Labeled data saved to 'MrBeast_video_comments_labeled.csv'
